# Project Stage - III (Basic Machine Learning)

### Import essential libraries

In [109]:
import os
# Science libraries
import pandas as pd
import numpy as np
# Ploting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
# Machine learning
from sklearn.linear_model import LinearRegression

In [110]:
# Create folder to hold the graphs
if not os.path.exists("images"):
    os.mkdir("images")

### Read and Filtter Dataset

#### Read dataset

In [115]:
# Read super Covid-19 dataframe with state as index. 
covid = pd.read_csv("../../data/output/covid.csv", index_col=0)

#### Clean the dataset.
remove unnessasery columns or rows.

In [116]:
# Drop unneccesary variables in Super covid-19 dataframe. 
covid.drop(["countyFIPS","stateFIPS"], axis=1, inplace=True)

# Drop unneccesary rows in Super COVID-19 dataframe.
covid.drop(
    covid.loc[covid['County Name'] == ("Statewide Unallocated" or 'New York City Unallocated')].index,
    inplace=True)

# Reset indexes after removing rows
covid.reset_index(drop=True, inplace=True)

#### Splite super covid dataset
Create dataframe that hold covid death and dataframe that hold covid cases.

In [117]:
# Regalare expression that will help filltering the data.
regex_cases = '(^[0-9]+[/]+[0-9]+[/]+[0-9]+[_]+[x])|^County Name$|^State$|^population$'
regex_deaths = '(^[0-9]+[/]+[0-9]+[/]+[0-9]+[_]+[y])|^County Name$|^State$|^population$'

cases = covid.filter(regex=regex_cases)
deaths = covid.filter(regex=regex_deaths)

Save the first three columns temprary so that we fix:
1. the date syntax.
2. Find the new cases/deaths

In [284]:
# Split the data: first three columns
covid_columns = covid[covid.columns[:3]]

# Split the data: fourth to the end columns
cases_data = cases[cases.columns[3:]]
deaths_data = deaths[deaths.columns[3:]]

#### Convert date type from String to Timestamp

In [285]:
# Fixing date string syntax
daily_cases = cases_data.rename(
    columns=lambda x: (pd.to_datetime((x.split('_')[0]), errors='ignore')),
    errors='raise')
daily_deaths = deaths_data.rename(
    columns=lambda x: (pd.to_datetime((x.split('_')[0]), errors='ignore')),
    errors='raise')

In [286]:
daily_cases

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16
0,0,0,0,0,0,0,0,0,0,0,...,1852,1863,1882,1898,1905,1911,1924,1928,1949,1966
1,0,0,0,0,0,0,0,0,0,0,...,6134,6141,6172,6190,6203,6220,6248,6270,6285,6333
2,0,0,0,0,0,0,0,0,0,0,...,927,927,939,942,942,944,950,950,965,968
3,0,0,0,0,0,0,0,0,0,0,...,703,708,719,726,736,738,744,744,761,771
4,0,0,0,0,0,0,0,0,0,0,...,1673,1681,1689,1704,1713,1722,1742,1750,1768,1783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,0,0,0,0,0,0,0,0,0,0,...,378,382,393,394,394,394,401,402,406,410
3141,0,0,0,0,0,0,0,0,0,0,...,633,645,656,658,662,675,679,686,687,692
3142,0,0,0,0,0,0,0,0,0,0,...,368,373,378,379,380,381,384,385,388,392
3143,0,0,0,0,0,0,0,0,0,0,...,119,123,123,124,125,126,127,132,132,133


#### Find the daily new cases/death.
Calculate the diffrence between the current day and the day before it. The results is the new cases.

In [287]:
daily_cases = daily_cases.diff(axis=1).fillna(0)
daily_deaths = daily_deaths.diff(axis=1).fillna(0)

#### Merge the popolation, states and county names to the daily cases/deaths

In [288]:
new_cases = covid_columns.merge(daily_cases, left_index=True, right_index=True)
new_deaths = covid_columns.merge(daily_deaths, left_index=True, right_index=True)

In [289]:
new_cases.head()

,County Name,State,population,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,...,2020-10-07 00:00:00,2020-10-08 00:00:00,2020-10-09 00:00:00,2020-10-10 00:00:00,2020-10-11 00:00:00,2020-10-12 00:00:00,2020-10-13 00:00:00,2020-10-14 00:00:00,2020-10-15 00:00:00,2020-10-16 00:00:00
0,Autauga County,AL,55869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.0,11.0,19.0,16.0,7.0,6.0,13.0,4.0,21.0,17.0
1,Baldwin County,AL,223234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.0,7.0,31.0,18.0,13.0,17.0,28.0,22.0,15.0,48.0
2,Barbour County,AL,24686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,12.0,3.0,0.0,2.0,6.0,0.0,15.0,3.0
3,Bibb County,AL,22394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,5.0,11.0,7.0,10.0,2.0,6.0,0.0,17.0,10.0
4,Blount County,AL,57826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,8.0,8.0,15.0,9.0,9.0,20.0,8.0,18.0,15.0


In [290]:
new_deaths.head()

,County Name,State,population,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,...,2020-10-07 00:00:00,2020-10-08 00:00:00,2020-10-09 00:00:00,2020-10-10 00:00:00,2020-10-11 00:00:00,2020-10-12 00:00:00,2020-10-13 00:00:00,2020-10-14 00:00:00,2020-10-15 00:00:00,2020-10-16 00:00:00
0,Autauga County,AL,55869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Baldwin County,AL,223234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,8.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,Barbour County,AL,24686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bibb County,AL,22394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Blount County,AL,57826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0


## Linear regression models for predicting cases and deaths in US.

#### United States Cases

Sum up all states daily cases

In [360]:
united_states_cases = pd.DataFrame(new_cases[new_cases.columns[3:]].sum(axis=0)).T

In [361]:
united_states_cases

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16
0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,1.0,0.0,1.0,...,49901.0,56462.0,56731.0,54111.0,43207.0,42554.0,50399.0,59692.0,66735.0,63931.0


Fit a linewar regression model using sklearn's LinearRegression package

In [362]:
X, Y = united_states_cases.T, united_states_cases.columns

mdl = LinearRegression().fit(X,Y)
result = mdl.predict(X)
m = mdl.coef_[0]
b = mdl.intercept_
print(f'formula: y= {m}x + {b}')

formula: y= 267151945867.48468x + 1.5833451689194383e+18


In [363]:
df = united_states_cases.T.reset_index()
fig = px.scatter(df, x="index", y=0, trendline="ols")
fig.show()


In [349]:
# plt.scatter(X.index,X, color='blue')
# plt.plot([0,X.size], [b,m*X.size+b], 'o')
trace0 = go.Scatter(x=X.index, y=X[0],  mode='markers', name="United States")
trace1 = go.Scatter(x=X.index, y=result,  mode='lines', name="Line")

mydata = [trace0, trace1]
# mydata = [trace0]

mylayout = go.Layout(
    title="States Weekly Cases"
)

fig = go.Figure(data=mydata, layout=mylayout)
fig.show()

# plotly.offline.iplot(fig, filename = '')

# fig.write_image("images/fig1.png")

## Non-Linear regression models for predicting cases and deaths in US.